# CB. Контентные рекомендации. Получение векторов товаров

In [ ]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import polars as pl
import dask.dataframe as dd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import pyarrow.parquet as pq
import os

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### Категории

#### Проход по всем parquet ver 2.0

In [3]:
# Получаем полный список файлов
items_files = glob.glob('data/ml_ozon_recsys_train_final_apparel_items_data/*.parquet')

# Читаем все файлы явно указывая список
ddf = dd.read_parquet(items_files, columns=['catalogid'])

# Проверяем количество файлов
print(f"Загружено файлов: {len(items_files)}")
print(f"Размер DataFrame: {len(ddf):,} строк")

# Считаем уникальные catalogid
catalogid_counter = (
    ddf['catalogid']
    .value_counts()
    .compute()
    .reset_index(name='count')
    .rename(columns={'index': 'catalogid'})
)

Загружено файлов: 100
Размер DataFrame: 6,439,422 строк


In [5]:
catalogid_counter.sort_values(by='count')

,catalogid,count
48,35119,1
101,32132,1
366,9020,1
244,11237,1
23,7646000,1
...,...,...
87,7502,199919
330,7660,219858
128,36484,225336
129,7508,267444


##### Функции

In [30]:
# Конфигурация
KNOWN_CATALOG_IDS = catalogid_counter['catalogid'].sort_values().tolist()  # Заполните известными catalogid
INPUT_FILES = glob.glob('data/ml_ozon_recsys_train_final_apparel_items_data/*.parquet')
TEMP_DIR = './catalog_tmp'
OUTPUT_DIR = './result_vectors'
BATCH_SIZE = 5000  # Размер батча для обработки
TEXT_DIM_REDUCTION = 50  # Размерность после уменьшения для текстов
EMBED_DIM_REDUCTION = 50  # Размерность после уменьшения для эмбеддингов
LARGE_CATALOG_THRESHOLD = 50000  # Порог для определения больших catalogid

In [ ]:
# Создаем директории для временных файлов по каждому catalogid
os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 1. Функция для обработки атрибутов
def process_attributes(df):
    """Разворачиваем attributes в отдельные столбцы"""
    attrs_list = []
    for attr_list in df['attributes']:
        attrs = {}
        for d in attr_list:
            name = d.get('attribute_name')
            value = d.get('attribute_value')
            if name in ['Annotation', 'Type', 'Brand', 'ColorBase', 'Material']:
                attrs[name] = value
        attrs_list.append(attrs)
    
    attrs_df = pd.DataFrame(attrs_list)
    attrs_df = attrs_df.rename(columns={'Annotation': 'annotation'})
    return pd.concat([df[['item_id', 'itemname', 'fclip_embed', 'catalogid']], attrs_df], axis=1)

# 2. Предварительная обработка и распределение по catalogid
def preprocess_and_distribute():
    """Обрабатываем файлы и распределяем данные по catalogid"""
    for file_path in tqdm(INPUT_FILES, desc="Обработка файлов"):
        print(f"Обрабатываем файл: {file_path}")
        
        # Читаем файл по частям
        parquet_file = pq.ParquetFile(file_path)
        
        for i in range(parquet_file.num_row_groups):
            # Читаем одну группу строк
            table = parquet_file.read_row_group(i)
            df = table.to_pandas()
            
            # Обрабатываем атрибуты
            df_processed = process_attributes(df)
            
            # Сохраняем по catalogid
            for catalog_id in KNOWN_CATALOG_IDS:
                catalog_data = df_processed[df_processed['catalogid'] == catalog_id]
                if not catalog_data.empty:
                    output_path = f"{TEMP_DIR}/{catalog_id}.parquet"
                    
                    # Если файл уже существует, добавляем к нему
                    if os.path.exists(output_path):
                        existing_data = pd.read_parquet(output_path)
                        combined = pd.concat([existing_data, catalog_data], ignore_index=True)
                        combined.to_parquet(output_path, index=False)
                    else:
                        catalog_data.to_parquet(output_path, index=False)


In [ ]:
# 1. Обработка небольших catalogid с проверкой размерности
def process_small_catalog(catalog_id, df):
    """Обрабатываем небольшие catalogid целиком с проверкой размерности"""
    print(f"Обрабатываем небольшой catalogid: {catalog_id} (размер: {len(df)})")
    
    # Инициализируем преобразователи
    text_vectorizer = TfidfVectorizer(max_features=5000)
    scaler = StandardScaler()
    
    # Подготавливаем категориальные признаки (без Size)
    cat_columns = ['Type', 'Brand', 'ColorBase', 'Material']  # Убрали 'Size'
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    ohe.fit(df[cat_columns].fillna('UNK'))
    
    # Текстовые признаки
    name_vec = text_vectorizer.fit_transform(df['itemname'].fillna(''))
    desc_vec = text_vectorizer.fit_transform(df['annotation'].fillna(''))
    
    # Уменьшение размерности текстов с проверкой
    # Для name_vec
    n_components_name = min(TEXT_DIM_REDUCTION, name_vec.shape[1], name_vec.shape[0]-1)
    if n_components_name > 0:
        svd_name = TruncatedSVD(n_components=n_components_name)
        name_reduced = svd_name.fit_transform(name_vec)
    else:
        name_reduced = np.zeros((name_vec.shape[0], 1))
        print(f"Предупреждение: name_vec имеет недостаточно признаков для SVD")
    
    # Для desc_vec
    n_components_desc = min(TEXT_DIM_REDUCTION, desc_vec.shape[1], desc_vec.shape[0]-1)
    if n_components_desc > 0:
        svd_desc = TruncatedSVD(n_components=n_components_desc)
        desc_reduced = svd_desc.fit_transform(desc_vec)
    else:
        desc_reduced = np.zeros((desc_vec.shape[0], 1))
        print(f"Предупреждение: desc_vec имеет недостаточно признаков для SVD")
    
    # Кодирование категориальных признаков
    cat_encoded = ohe.transform(df[cat_columns].fillna('UNK'))
    
    # Обработка эмбеддингов с проверкой размерности
    all_embeddings = np.vstack(df['fclip_embed'].values)
    
    # Для PCA нужно: n_components <= min(n_samples, n_features)
    n_components_embed = min(
        EMBED_DIM_REDUCTION, 
        all_embeddings.shape[0]-1,  # n_samples-1
        all_embeddings.shape[1]     # n_features
    )
    
    if n_components_embed > 0:
        pca = PCA(n_components=n_components_embed)
        embed_reduced = pca.fit_transform(all_embeddings)
    else:
        embed_reduced = np.zeros((all_embeddings.shape[0], 1))
        print(f"Предупреждение: эмбеддинги имеют недостаточно признаков для PCA")
    
    # Объединение всех признаков
    final_vectors = np.hstack([
        cat_encoded,
        name_reduced * 0.5,  # Веса для текстовых features
        desc_reduced * 0.3,
        embed_reduced
    ])
    
    # Нормализация
    final_vectors_normalized = scaler.fit_transform(final_vectors)
    
    # Сохранение результатов
    result_df = pd.DataFrame({
        'item_id': df['item_id'].values,
        'vector': list(final_vectors_normalized)
    })
    result_df.to_parquet(f"{OUTPUT_DIR}/{catalog_id}_vectors.parquet", index=False)
    
    # Сохраняем scaler для последующего использования
    np.save(f"{OUTPUT_DIR}/{catalog_id}_scaler_mean.npy", scaler.mean_)
    np.save(f"{OUTPUT_DIR}/{catalog_id}_scaler_scale.npy", scaler.scale_)

# 2. Обработка больших catalogid с проверкой размерности
def process_large_catalog(catalog_id, df):
    """Обрабатываем большие catalogid батчами с проверкой размерности"""
    print(f"Обрабатываем большой catalogid: {catalog_id} (размер: {len(df)})")
    
    # Сначала соберем всю статистику для обучения преобразователей
    print("Сбор статистики для обучения преобразователей...")
    
    # Для текстовых признаков - соберем все тексты для обучения TF-IDF
    all_texts = pd.concat([
        df['itemname'].fillna(''),
        df['annotation'].fillna('')
    ])
    
    # Для категориальных признаков (без Size)
    cat_columns = ['Type', 'Brand', 'ColorBase', 'Material']  # Убрали 'Size'
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    ohe.fit(df[cat_columns].fillna('UNK'))
    
    # Для эмбеддингов - соберем все эмбеддинги для обучения PCA
    all_embeddings = np.vstack(df['fclip_embed'].values)
    
    # Обучаем преобразователи на всех данных
    print("Обучение преобразователей...")
    text_vectorizer = TfidfVectorizer(max_features=5000)
    text_vectorizer.fit(all_texts)
    
    # Проверяем размерность для SVD
    sample_size = min(10000, len(df))
    sample_texts = all_texts.sample(sample_size, random_state=42)
    sample_text_vectors = text_vectorizer.transform(sample_texts)
    
    n_components_svd = min(
        TEXT_DIM_REDUCTION, 
        sample_text_vectors.shape[1], 
        sample_text_vectors.shape[0]-1
    )
    if n_components_svd > 0:
        svd = TruncatedSVD(n_components=n_components_svd)
        svd.fit(sample_text_vectors)
    else:
        svd = None
        print(f"Предупреждение: недостаточно признаков для SVD (n_features={sample_text_vectors.shape[1]})")
    
    # Проверяем размерность для PCA
    n_components_pca = min(
        EMBED_DIM_REDUCTION, 
        all_embeddings.shape[0]-1,  # n_samples-1
        all_embeddings.shape[1]     # n_features
    )
    if n_components_pca > 0:
        pca = PCA(n_components=n_components_pca)
        pca.fit(all_embeddings)
    else:
        pca = None
        print(f"Предупреждение: недостаточно признаков для PCA (n_features={all_embeddings.shape[1]})")
    
    # Определяем размерность финальных векторов
    print("Определение размерности финальных векторов...")
    test_sample = df.iloc[:1].copy()
    
    # Обрабатываем тестовый sample для определения размерности
    name_vec_test = text_vectorizer.transform(test_sample['itemname'].fillna(''))
    desc_vec_test = text_vectorizer.transform(test_sample['annotation'].fillna(''))
    
    # Применяем SVD если доступен
    if svd is not None:
        name_reduced_test = svd.transform(name_vec_test)
        desc_reduced_test = svd.transform(desc_vec_test)
    else:
        name_reduced_test = np.zeros((name_vec_test.shape[0], 1))
        desc_reduced_test = np.zeros((desc_vec_test.shape[0], 1))
    
    cat_encoded_test = ohe.transform(test_sample[cat_columns].fillna('UNK'))
    
    # Применяем PCA если доступен
    if pca is not None:
        embed_reduced_test = pca.transform(np.vstack(test_sample['fclip_embed'].values))
    else:
        embed_reduced_test = np.zeros((len(test_sample), 1))
    
    final_vector_test = np.hstack([
        cat_encoded_test,
        name_reduced_test * 0.5,  # Веса для текстовых features
        desc_reduced_test * 0.3,
        embed_reduced_test
    ])
    
    n_features = final_vector_test.shape[1]
    n_samples = len(df)
    
    print(f"Размерность финальных векторов: {n_features}")
    
    # ПЕРВЫЙ ПРОХОД: вычисляем среднее и std для нормализации
    print("Первый проход: вычисление статистики для нормализации...")
    sum_vectors = np.zeros(n_features, dtype='float64')
    sum_sq_vectors = np.zeros(n_features, dtype='float64')
    count = 0
    
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Сбор статистики"):
        batch = df.iloc[i:i+BATCH_SIZE].copy()
        
        # Текстовые признаки
        name_vec = text_vectorizer.transform(batch['itemname'].fillna(''))
        desc_vec = text_vectorizer.transform(batch['annotation'].fillna(''))
        
        # Уменьшение размерности текстов
        if svd is not None:
            name_reduced = svd.transform(name_vec)
            desc_reduced = svd.transform(desc_vec)
        else:
            name_reduced = np.zeros((name_vec.shape[0], 1))
            desc_reduced = np.zeros((desc_vec.shape[0], 1))
        
        # Кодирование категориальных признаков
        cat_encoded = ohe.transform(batch[cat_columns].fillna('UNK'))
        
        # Обработка эмбеддингов
        if pca is not None:
            embed_reduced = pca.transform(np.vstack(batch['fclip_embed'].values))
        else:
            embed_reduced = np.zeros((len(batch), 1))
        
        # Объединение всех признаков
        final_vectors = np.hstack([
            cat_encoded,
            name_reduced * 0.5,  # Веса для текстовых features
            desc_reduced * 0.3,
            embed_reduced
        ]).astype('float32')
        
        # Аккумулируем статистику
        batch_sum = np.sum(final_vectors, axis=0)
        batch_sum_sq = np.sum(final_vectors ** 2, axis=0)
        
        # Обновляем общую статистику
        sum_vectors += batch_sum
        sum_sq_vectors += batch_sum_sq
        count += len(final_vectors)
    
    # Вычисляем среднее и стандартное отклонение
    mean = sum_vectors / count
    std = np.sqrt((sum_sq_vectors / count) - (mean ** 2))
    std = np.where(std < 1e-8, 1.0, std)  # Избегаем деления на ноль
    
    # Сохраняем параметры нормализации
    np.save(f"{OUTPUT_DIR}/{catalog_id}_normalization_mean.npy", mean.astype('float32'))
    np.save(f"{OUTPUT_DIR}/{catalog_id}_normalization_std.npy", std.astype('float32'))
    
    # ВТОРОЙ ПРОХОД: нормализация и сохранение
    print("Второй проход: нормализация и сохранение...")
    
    # Создаем memory-mapped файлы для финальных векторов и идентификаторов
    vectors_path = f"{OUTPUT_DIR}/{catalog_id}_vectors.npy"
    item_ids_path = f"{OUTPUT_DIR}/{catalog_id}_item_ids.npy"
    metadata_path = f"{OUTPUT_DIR}/{catalog_id}_metadata.json"
    
    vectors_mmap = np.memmap(vectors_path, dtype='float32', mode='w+', 
                            shape=(n_samples, n_features))
    item_ids_mmap = np.memmap(item_ids_path, dtype='int64', mode='w+', 
                             shape=(n_samples,))
    
    # Сохраняем метаданные
    metadata = {
        'n_samples': n_samples,
        'n_features': n_features,
        'catalog_id': catalog_id,
        'dtype': 'float32',
        'normalized': True,
        'mean_path': f"{catalog_id}_normalization_mean.npy",
        'std_path': f"{catalog_id}_normalization_std.npy"
    }
    
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f)
    
    # Обрабатываем данные батчами, нормализуем и сохраняем
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Нормализация и сохранение"):
        batch = df.iloc[i:i+BATCH_SIZE].copy()
        end_idx = min(i + BATCH_SIZE, len(df))
        
        # Текстовые признаки
        name_vec = text_vectorizer.transform(batch['itemname'].fillna(''))
        desc_vec = text_vectorizer.transform(batch['annotation'].fillna(''))
        
        # Уменьшение размерности текстов
        if svd is not None:
            name_reduced = svd.transform(name_vec)
            desc_reduced = svd.transform(desc_vec)
        else:
            name_reduced = np.zeros((name_vec.shape[0], 1))
            desc_reduced = np.zeros((desc_vec.shape[0], 1))
        
        # Кодирование категориальных признаков
        cat_encoded = ohe.transform(batch[cat_columns].fillna('UNK'))
        
        # Обработка эмбеддингов
        if pca is not None:
            embed_reduced = pca.transform(np.vstack(batch['fclip_embed'].values))
        else:
            embed_reduced = np.zeros((len(batch), 1))
        
        # Объединение всех признаков
        final_vectors = np.hstack([
            cat_encoded,
            name_reduced * 0.5,  # Веса для текстовых features
            desc_reduced * 0.3,
            embed_reduced
        ]).astype('float32')
        
        # НОРМАЛИЗАЦИЯ: (x - mean) / std
        final_vectors_normalized = (final_vectors - mean) / std
        
        # Записываем в memory-mapped arrays
        vectors_mmap[i:end_idx] = final_vectors_normalized
        item_ids_mmap[i:end_idx] = batch['item_id'].values
        
        # Принудительно записываем изменения на диск
        vectors_mmap.flush()
        item_ids_mmap.flush()
    
    # Закрываем memory-mapped arrays
    del vectors_mmap
    del item_ids_mmap
    
    print(f"Обработка с нормализацией завершена. Векторы сохранены в: {vectors_path}")

# 3. Основной пайплайн обработки
def process_catalog(catalog_id):
    """Обрабатываем данные для конкретного catalogid с проверкой размерности"""
    print(f"Обрабатываем catalogid: {catalog_id}")
    
    input_path = f"{TEMP_DIR}/{catalog_id}.parquet"
    if not os.path.exists(input_path):
        print(f"Нет данных для catalogid: {catalog_id}")
        return
    
    # Читаем данные
    df = pd.read_parquet(input_path)
    
    if len(df) > LARGE_CATALOG_THRESHOLD:
        process_large_catalog(catalog_id, df)
    else:
        process_small_catalog(catalog_id, df)

##### проход

In [32]:
# Этап 1: Предобработка и распределение по catalogid
print("Начинаем предобработку и распределение данных...")
# preprocess_and_distribute()

# Этап 2: Обработка каждого catalogid
print("Начинаем обработку каждого catalogid...")
for catalog_id in tqdm(KNOWN_CATALOG_IDS, desc="Обработка catalogid"):
    process_catalog(catalog_id)

print("Обработка завершена!")

Начинаем предобработку и распределение данных...
Начинаем обработку каждого catalogid...


Обработка catalogid:   0%|          | 0/478 [00:00<?, ?it/s]

Обрабатываем catalogid: 1652
Обрабатываем небольшой catalogid: 1652 (размер: 850)


Обработка catalogid:   0%|          | 1/478 [00:00<02:54,  2.73it/s]

Обрабатываем catalogid: 1657
Обрабатываем небольшой catalogid: 1657 (размер: 10603)


...

Обработка catalogid:  95%|█████████▍| 454/478 [27:20<00:09,  2.60it/s]

Обрабатываем catalogid: 30613000
Обрабатываем небольшой catalogid: 30613000 (размер: 43)
Обрабатываем catalogid: 31311000
Обрабатываем небольшой catalogid: 31311000 (размер: 4)
Обрабатываем catalogid: 31577000
Обрабатываем небольшой catalogid: 31577000 (размер: 42)
Обрабатываем catalogid: 33331000
Обрабатываем небольшой catalogid: 33331000 (размер: 24)
Обрабатываем catalogid: 36410000
Обрабатываем небольшой catalogid: 36410000 (размер: 30)


Обработка catalogid:  96%|█████████▌| 458/478 [27:20<01:11,  3.58s/it]


Обрабатываем catalogid: 36471000
Обрабатываем небольшой catalogid: 36471000 (размер: 21)
Обрабатываем catalogid: 36683000
Обрабатываем небольшой catalogid: 36683000 (размер: 31)
Обрабатываем catalogid: 36697000
Обрабатываем небольшой catalogid: 36697000 (размер: 1)


ValueError: empty vocabulary; perhaps the documents only contain stop words

#### Открываем npz от items по подкатегориям

#### Функции записи ближайших соседей item_ids

In [2]:
OUTPUT_DIR = './result_vectors/1'
NEIGHBORS_DIR = './nearest_neighbors'
TOP_K = 50

# Создаем директории
os.makedirs(NEIGHBORS_DIR, exist_ok=True)

class NumpyEncoder(json.JSONEncoder):
    """Кастомный JSON encoder для обработки numpy типов"""
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int32, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float32, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.bool_):
            return bool(obj)
        return super().default(obj)

def load_vectors(catalog_id):
    """Загружает векторы для catalogid"""
    vectors_path = f"{OUTPUT_DIR}/{catalog_id}_vectors.npy"
    item_ids_path = f"{OUTPUT_DIR}/{catalog_id}_item_ids.npy"
    metadata_path = f"{OUTPUT_DIR}/{catalog_id}_metadata.json"
    
    if os.path.exists(vectors_path) and os.path.exists(item_ids_path):
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
        
        vectors = np.memmap(vectors_path, dtype=metadata['dtype'], mode='r',
                           shape=(metadata['n_samples'], metadata['n_features']))
        item_ids = np.memmap(item_ids_path, dtype='int64', mode='r',
                            shape=(metadata['n_samples'],))
        
        # Конвертируем в обычные numpy arrays для обработки в памяти
        vectors = np.array(vectors)
        item_ids = np.array(item_ids)
        
        return vectors, item_ids
    
    parquet_path = f"{OUTPUT_DIR}/{catalog_id}_vectors.parquet"
    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
        vectors = np.vstack(df['vector'].values)
        item_ids = df['item_id'].values
        return vectors, item_ids
    
    raise FileNotFoundError(f"Векторы для catalogid {catalog_id} не найдены")

def split_data(vectors, item_ids):
    """Разделяет данные пополам"""
    n_samples = len(vectors)
    mid = n_samples // 2
    
    vectors_1, item_ids_1 = vectors[:mid], item_ids[:mid]
    vectors_2, item_ids_2 = vectors[mid:], item_ids[mid:]
    
    return vectors_1, item_ids_1, vectors_2, item_ids_2

def clean_vectors(vectors, item_ids):
    """Очищает векторы от NaN значений и фильтрует валидные данные"""
    # Проверяем на NaN значения
    valid_mask = ~np.isnan(vectors).any(axis=1)
    
    if not np.any(valid_mask):
        print("Все векторы содержат NaN значения")
        return None, None
    
    if np.sum(~valid_mask) > 0:
        print(f"Удалено {np.sum(~valid_mask)} векторов с NaN значениями")
    
    clean_vectors = vectors[valid_mask]
    clean_item_ids = item_ids[valid_mask]
    
    return clean_vectors, clean_item_ids

def find_nearest_neighbors(catalog_id, vectors, item_ids):
    """Находит ближайших соседей по косинусной близости"""
    print(f"Поиск ближайших соседей для catalogid: {catalog_id} (размер: {len(vectors)})")
    
    # Очищаем векторы от NaN
    vectors_clean, item_ids_clean = clean_vectors(vectors, item_ids)
    
    if vectors_clean is None or len(vectors_clean) == 0:
        print(f"Нет валидных векторов для catalogid {catalog_id}")
        return {}
    
    if len(vectors_clean) <= TOP_K + 1:
        print(f"Маленький catalogid {catalog_id}, используем попарное сравнение")
        return find_neighbors_small_catalog(vectors_clean, item_ids_clean, catalog_id)
    
    # Нормализуем векторы для косинусного сходства
    norms = np.linalg.norm(vectors_clean, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Избегаем деления на ноль
    vectors_normalized = vectors_clean / norms
    
    # Используем NearestNeighbors для эффективного поиска
    n_neighbors = min(TOP_K + 1, len(vectors_clean))
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute')
    nbrs.fit(vectors_normalized)
    
    # Находим ближайших соседей
    distances, indices = nbrs.kneighbors(vectors_normalized)
    
    # Создаем словарь с результатами
    neighbors_dict = {}
    
    for i in tqdm(range(len(vectors_clean)), desc=f"Обработка товаров {catalog_id}"):
        # Пропускаем самого себя (первый элемент - сам товар)
        neighbor_indices = indices[i][1:n_neighbors]
        neighbor_distances = 1 - distances[i][1:n_neighbors]  # cosine distance -> similarity
        
        neighbor_item_ids = item_ids_clean[neighbor_indices]
        
        # Конвертируем numpy типы в стандартные Python типы
        item_id_key = str(int(item_ids_clean[i]))  # Гарантируем, что ключ - строка
        neighbors_dict[item_id_key] = {
            'neighbors': [int(x) for x in neighbor_item_ids],  # Конвертируем в int
            'similarities': [float(x) for x in neighbor_distances],  # Конвертируем в float
            'catalog_id': catalog_id
        }
    
    return neighbors_dict

def find_neighbors_small_catalog(vectors, item_ids, catalog_id):
    """Поиск соседей для маленьких catalogid через попарное сравнение"""
    print(f"Попарное сравнение для маленького catalogid {catalog_id}")
    
    if len(vectors) == 1:
        # Если только один вектор, возвращаем пустой список соседей
        return {str(int(item_ids[0])): {
            'neighbors': [],
            'similarities': [],
            'catalog_id': catalog_id
        }}
    
    # Вычисляем попарные косинусные схожести
    similarities = cosine_similarity(vectors)
    np.fill_diagonal(similarities, -1)  # Исключаем самосравнение
    
    neighbors_dict = {}
    
    for i in range(len(vectors)):
        # Получаем индексы TOP_K самых похожих товаров
        n_neighbors = min(TOP_K, len(vectors) - 1)  # Не больше чем доступно соседей
        top_indices = np.argsort(similarities[i])[-n_neighbors:][::-1]
        top_similarities = similarities[i][top_indices]
        top_item_ids = item_ids[top_indices]
        
        item_id_key = str(int(item_ids[i]))  # Гарантируем, что ключ - строка
        neighbors_dict[item_id_key] = {
            'neighbors': [int(x) for x in top_item_ids],  # Конвертируем в int
            'similarities': [float(x) for x in top_similarities],  # Конвертируем в float
            'catalog_id': catalog_id
        }
    
    return neighbors_dict

def save_neighbors(catalog_id, neighbors_dict):
    """Сохраняет результаты поиска соседей"""
    if not neighbors_dict:
        print(f"Нет данных для сохранения catalogid {catalog_id}")
        return
    
    # Сохраняем в JSON с кастомным encoder'ом
    output_path = f"{NEIGHBORS_DIR}/{catalog_id}_neighbors.json"
    
    with open(output_path, 'w') as f:
        json.dump(neighbors_dict, f, cls=NumpyEncoder, indent=2)
    
    # Также сохраняем в parquet для аналитики
    parquet_data = []
    for item_id_str, data in neighbors_dict.items():
        item_id = int(item_id_str)
        for i, (neighbor_id, similarity) in enumerate(zip(data['neighbors'], data['similarities'])):
            parquet_data.append({
                'catalog_id': catalog_id,
                'item_id': item_id,
                'neighbor_item_id': neighbor_id,
                'similarity_score': similarity,
                'rank': i + 1
            })
    
    if parquet_data:
        df = pd.DataFrame(parquet_data)
        df.to_parquet(f"{NEIGHBORS_DIR}/{catalog_id}_neighbors.parquet", index=False)
    
    print(f"Сохранено {len(neighbors_dict)} записей для catalogid {catalog_id}")

def process_catalog_neighbors(catalog_id):
    """Обрабатывает поиск соседей для одного catalogid"""
    try:
        print(f"Обрабатываем catalogid: {catalog_id}")
        
        # Загружаем векторы
        vectors, item_ids = load_vectors(catalog_id)
        vectors_1, item_ids_1, vectors_2, item_ids_2 = split_data(vectors, item_ids)
        # if len(vectors) == 0:
        #     print(f"Пустой catalogid: {catalog_id}")
        #     return False
        
        # Находим ближайших соседей
        neighbors_dict_1 = find_nearest_neighbors(catalog_id, vectors_1, item_ids_1)
        
        # Сохраняем результаты
        save_neighbors(catalog_id, neighbors_dict_1)
        neighbors_dict_2 = find_nearest_neighbors(catalog_id, vectors_2, item_ids_2)
        catalog_id = catalog_id + '_2'
        save_neighbors(catalog_id, neighbors_dict_2)
        
        return True
        
    except Exception as e:
        print(f"Ошибка при обработке catalogid {catalog_id}: {e}")
        import traceback
        traceback.print_exc()
        return False

def create_index_files():
    """Создает индексные файлы для быстрого доступа"""
    print("Создание индексных файлов...")
    
    index_data = {}
    
    for filename in os.listdir(NEIGHBORS_DIR):
        if filename.endswith('_neighbors.json'):
            catalog_id = filename.replace('_neighbors.json', '')
            file_path = f"{NEIGHBORS_DIR}/{filename}"
            
            try:
                # Получаем статистику файла
                file_size = os.path.getsize(file_path)
                with open(file_path, 'r') as f:
                    data = json.load(f)
                    item_count = len(data)
                
                index_data[catalog_id] = {
                    'neighbors_file': filename,
                    'parquet_file': filename.replace('.json', '.parquet'),
                    'file_size_mb': round(file_size / (1024 * 1024), 2),
                    'item_count': item_count
                }
            except Exception as e:
                print(f"Ошибка при обработке файла {filename}: {e}")
                continue
    
    # Сохраняем индекс
    with open(f"{NEIGHBORS_DIR}/index.json", 'w') as f:
        json.dump(index_data, f, indent=2)
    
    print("Индексные файлы созданы")

def get_neighbors(catalog_id, item_id):
    """Функция для получения ближайших соседей по запросу"""
    neighbors_file = f"{NEIGHBORS_DIR}/{catalog_id}_neighbors.json"
    
    if not os.path.exists(neighbors_file):
        return None
    
    try:
        with open(neighbors_file, 'r') as f:
            neighbors_data = json.load(f)
        
        item_id_str = str(item_id)
        return neighbors_data.get(item_id_str)
    
    except json.JSONDecodeError as e:
        print(f"Ошибка чтения JSON файла {neighbors_file}: {e}")
        return None
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return None

# Дополнительные утилиты
def analyze_catalog(catalog_id):
    """Анализирует catalogid и показывает статистику"""
    vectors, item_ids = load_vectors(catalog_id)
    print(f"Catalog {catalog_id}: {len(vectors)} векторов")
    
    if len(vectors) > 0:
        # Проверяем на NaN
        nan_count = np.isnan(vectors).any(axis=1).sum()
        print(f"Векторов с NaN: {nan_count}")
        
        # Проверяем на нулевые векторы
        zero_count = np.all(vectors == 0, axis=1).sum()
        print(f"Нулевых векторов: {zero_count}")


In [ ]:
processed_catalogs = []
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith('_vectors.parquet') or filename.endswith('_vectors.npy'):
        catalog_id = filename.replace('_vectors.parquet', '').replace('_vectors.npy', '')
        if catalog_id not in processed_catalogs:
            processed_catalogs.append(catalog_id)

print(f"Найдено {len(processed_catalogs)} catalogid для обработки")

# Обрабатываем каждый catalogid
success_count = 0
for catalog_id in tqdm(processed_catalogs, desc="Поиск ближайших соседей"):
    if process_catalog_neighbors(catalog_id):
        success_count += 1

# Создаем индексные файлы
create_index_files()

print(f"Обработка завершена. Успешно обработано: {success_count}/{len(processed_catalogs)} catalogid")

Найдено 1 catalogid для обработки


Поиск ближайших соседей:   0%|          | 0/1 [00:00<?, ?it/s]

Обрабатываем catalogid: 7559
Поиск ближайших соседей для catalogid: 7559 (размер: 206818)


Обработка товаров 7559: 100%|██████████| 206818/206818 [00:03<00:00, 64206.47it/s]


Сохранено 197546 записей для catalogid 7559
Поиск ближайших соседей для catalogid: 7559 (размер: 206819)


Обработка товаров 7559: 100%|██████████| 206819/206819 [00:06<00:00, 32821.14it/s]


Сохранено 197537 записей для catalogid 7559_2


Поиск ближайших соседей: 100%|██████████| 1/1 [1:43:20<00:00, 6200.68s/it]


Создание индексных файлов...


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021998E89BE0>>
Traceback (most recent call last):
  File "c:\Users\UseR\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
  File "c:\Users\UseR\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1477, in enumerate
    def enumerate():
KeyboardInterrupt: 


In [21]:
items_df[items_df['catalogid'] == 7777]

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
47453,314869015,"Формодержатели для обуви, подпружиненные, две ...","[{'attribute_name': 'Type', 'attribute_value':...","[-0.20060442, -0.22965711, -0.13158977, 0.6714...",7777,266358188,33300984
66829,335138445,Колодки для обуви 2 шт,"[{'attribute_name': 'Type', 'attribute_value':...","[0.54418254, -0.97065365, -0.44515547, 0.47434...",7777,161703987,365171
81054,28764690,44-45р. Колодки для обуви деревянные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.5395479, -1.0622078, -0.48082113, 0.4074684...",7777,248376406,43171685
97225,250119073,"Колодки из красного кедра, р. 41/42, формодерж...","[{'attribute_name': 'Type', 'attribute_value':...","[0.44091186, -0.06559926, -0.28066403, 0.11426...",7777,241501565,37644112
152312,162115015,Мягкие формодержатели для обуви / Антизаломы и...,"[{'attribute_name': 'Type', 'attribute_value':...","[0.6210408, -1.0282655, -0.8357563, 0.16571093...",7777,195395764,17950766
165143,148289801,Колодки для обуви 1 шт,"[{'attribute_name': 'Type', 'attribute_value':...","[0.30978912, -0.9236857, -0.58002585, 0.188936...",7777,153866471,9048513
174299,177241601,Колодки для обуви 1 шт,"[{'attribute_name': 'Type', 'attribute_value':...","[0.0067255795, -1.3763212, -0.6828426, -0.0169...",7777,212554308,28986473
185290,12590245,"Растяжитель для обуви, винтовой, Колодки, Раст...","[{'attribute_name': 'Type', 'attribute_value':...","[0.12749772, -0.75848657, -0.43669933, -0.0377...",7777,251846990,14443245


In [ ]:
items_df

,item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
0,400301,Очки солнцезащитные,"[{'attribute_name': 'Type', 'attribute_value':...","[0.32023853, 0.11766258, -0.784552, 0.4168199,...",17080,243954236,33306429
1,678602,Футболка GOLDUSTIM Футболка женская,"[{'attribute_name': 'Type', 'attribute_value':...","[0.4360095, 0.6339668, 0.029741853, 0.09847632...",7508,8039633,27057398
2,746401,Худи Patagonia,"[{'attribute_name': 'Type', 'attribute_value':...","[0.47452784, -0.4727797, -0.51971936, -0.08221...",7555,120078071,11645051
3,872829,"Комплект носков adidas 3S C Spw Mid 3P, 3 пары","[{'attribute_name': 'Type', 'attribute_value':...","[-0.06987186, -2.2802832, -0.5829071, -0.33674...",36563,247120069,38088455
4,893255,Жилет Buy & Style жилеты,"[{'attribute_name': 'Type', 'attribute_value':...","[0.12488763, 0.5466293, 0.29031003, 0.21499962...",7535,327822613,19339929
...,...,...,...,...,...,...,...
643817,337511737,Брюки Koton,"[{'attribute_name': 'Type', 'attribute_value':...","[0.24482095, 0.9870064, 0.022553533, 0.534935,...",7512,27632331,38304950
643818,338025745,Брюки SELA,"[{'attribute_name': 'Type', 'attribute_value':...","[0.60598457, 0.7227131, 0.011865512, -0.307703...",36617,287124308,36965932
643819,338454800,Футболка,"[{'attribute_name': 'Type', 'attribute_value':...","[0.20186266, -1.0103929, -1.2348884, 0.5005319...",7508,290239692,32834089
643820,338596307,Платье Koton,"[{'attribute_name': 'Type', 'attribute_value':...","[0.071969345, 0.524616, -0.3169561, -0.290882,...",7502,319078691,24755854


In [ ]:
orders_df_delivered

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
0,15535,3423970,2025-01-11 19:20:02.550,delivered_orders,2025-01-16 10:53:03,2025-01-11
3,119241,2356390,2025-05-01 17:53:00.716,delivered_orders,2025-05-14 14:36:20,2025-05-01
5,119241,3470100,2025-05-15 23:54:20.860,delivered_orders,2025-05-17 15:34:40,2025-05-15
8,166547,815321,2025-02-12 19:09:39.360,delivered_orders,2025-02-14 18:04:32,2025-02-12
12,194001,3675391,2025-05-30 21:40:48.613,delivered_orders,2025-06-02 18:32:42,2025-05-30
...,...,...,...,...,...,...
18887113,339333163,2593920,2025-06-01 19:41:19.080,delivered_orders,2025-06-06 16:27:07,2025-06-01
18887114,339333163,522120,2025-03-31 05:52:03.046,delivered_orders,2025-04-07 07:27:51,2025-03-31
18887116,339333163,200191,2025-05-15 07:07:16.546,delivered_orders,2025-05-25 09:01:24,2025-05-15
18887118,339334813,2943711,2025-06-18 08:42:19.543,delivered_orders,2025-06-29 11:44:48,2025-06-18


In [ ]:
orders_df_delivered[orders_df_delivered['item_id'] == 5885425]

,item_id,user_id,created_timestamp,last_status,last_status_timestamp,created_date
552,5885425,4969760,2025-04-15 20:59:31.983,delivered_orders,2025-04-21 10:26:51,2025-04-15
